# Programming with Python

## Lecture 08: Class metaprogramming and memory management

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 12 Apr, 2025

## Class metaprogramming

This section is heavily influenced by the following:

*References:*

- Fluent Python, Luciano Ramalho

### Classes are objects

In [ ]:
class A:
    pass


class B(A):
    pass


class C(B):
    pass

In [ ]:
B.__bases__

In [ ]:
B.__subclasses__()

In [ ]:
B.mro()

### `type()`

The `type()` function in Python is used to determine the type of an object or dynamically create new classes.

In [ ]:
type(42)

In [ ]:
for t in int, str, list, set, tuple:
    print(type(t))

In [ ]:
class A:
    pass

type(A)

In [ ]:
type(ValueError)

In [ ]:
type(type)

`type` is a **metaclass**, meaning it is a class that builds other classes.

```
type(name, bases, attributes)
```

- `name`: Name of the class.
- `bases`: Tuple of base classes.
- `attributes`: Dictionary of attributes and methods.

In [ ]:
A = type("A", (object,), {"a": 42, "greet": lambda self: "Hello from A!"})
B = type("B", (A,), {"b": 24, "greet": lambda self: "Hello from B!"})

In [ ]:
a = A()
a.a, a.greet()

In [ ]:
b = B()
b.b, b.greet()

### Class factory function

In the following example we define a `record_factory` function that acts like `@dataclass`

In [ ]:
from typing import Union, Any
from collections.abc import Iterable, Iterator


FieldNames = str | Iterable[str]

def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()
    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')
    return tuple(names)


def record_factory(cls_name: str, field_names: FieldNames) -> type[tuple]:
    slots = parse_identifiers(field_names)

    def __init__(self, *args, **kwargs) -> None:
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ", ".join(f"{name}={value!r}"
            for name, value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f"{cls_name}({values})"

    cls_attrs = dict(
        __slots__=slots,
        __init__=__init__,
        __iter__=__iter__,
        __repr__=__repr__,
    )

    return type(cls_name, (object,), cls_attrs)

In [ ]:
Person = record_factory("Person", "name age")

In [ ]:
p = Person(name="John Doe", age=42)
p

In [ ]:
name, age = p

print(name)
print(age)

In [ ]:
p.name, p.age

### `__init_subclass__()` method

The `__init_subclass__()` method is a special class method that is automatically called whenever a subclass is created. It allows a base class to customize the behavior of its subclasses.

In [ ]:
int(), str(), list()

In [ ]:
from collections.abc import Callable
from typing import Any, NoReturn, get_type_hints


class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.constructor = constructor

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        instance.__dict__[self.name] = value

In [ ]:
class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)

    def __init_subclass__(subclass) -> None:
        super().__init_subclass__()
        for name, constructor in subclass._fields().items():
            setattr(subclass, name, Field(name, constructor))

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            self.__flag_unknown_attrs(*kwargs)

    def __setattr__(self, name: str, value: Any) -> None:
        if name in self._fields():
            cls = self.__class__
            descriptor = getattr(cls, name)
            descriptor.__set__(self, value)
        else:
            self.__flag_unknown_attrs(name)

    def __flag_unknown_attrs(self, *names: str) -> NoReturn:
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

In [ ]:
class Person(Checked):
    name: str
    age: int
    salary: float

In [ ]:
person = Person(name="John Doe", age=42, salary=199_999.99)
person

In [ ]:
person.name, person.age

In [ ]:
person.age = "text"

In [ ]:
person = Person()
person

In [ ]:
person = Person(first_name="John Doe", age=42, salary=199_999.99)

The `__init_subclass__()` method is called after the class is created. Adding `__slots__` to an existing class has no effect, meaning we cannot use the `__init_subclass__()` method for that purpose.

### Class decorator

A **class decorator** is a decorator that modifies or enhances a class instead of a function. It is similar to a function decorator but applies changes to a class rather than a function.

The main benefit of a class decorator over the simpler `__init_subclass__` is to avoid interfering with other class features, such as inheritance and metaclasses. This is actually why data classes are implemented as a class decorator.

In [ ]:
def _fields(cls: type) -> dict[str, type]:
    return get_type_hints(cls)

def __init__(self: Any, **kwargs: Any) -> None:
    for name in self._fields():
        value = kwargs.pop(name, ...)
        setattr(self, name, value)
    if kwargs:
        self.__flag_unknown_attrs(*kwargs)

def __setattr__(self: Any, name: str, value: Any) -> None:
    if name in self._fields():
        cls = self.__class__
        descriptor = getattr(cls, name)
        descriptor.__set__(self, value)
    else:
        self.__flag_unknown_attrs(name)

def __flag_unknown_attrs(self: Any, *names: str) -> NoReturn:
    plural = 's' if len(names) > 1 else ''
    extra = ', '.join(f'{name!r}' for name in names)
    cls_name = repr(self.__class__.__name__)
    raise AttributeError(f'{cls_name} has no attribute{plural} {extra}')

def _asdict(self: Any) -> dict[str, Any]:
    return {
        name: getattr(self, name)
        for name, attr in self.__class__.__dict__.items()
        if isinstance(attr, Field)
    }

def __repr__(self: Any) -> str:
    kwargs = ', '.join(
        f'{key}={value!r}' for key, value in self._asdict().items()
    )
    return f'{self.__class__.__name__}({kwargs})'

In [ ]:
def checked(cls: type) -> type:
    for name, constructor in _fields(cls).items():
        setattr(cls, name, Field(name, constructor))

    cls._fields = classmethod(_fields)

    instance_methods = (
        __init__,
        __repr__,
        __setattr__,
        _asdict,
        __flag_unknown_attrs,
    )
    for method in instance_methods:
        setattr(cls, method.__name__, method)

    return cls

In [ ]:
@checked
class Person:
    name: str
    age: int
    salary: float

In [ ]:
person = Person(name="John Doe", age=42, salary=199_999.99)
person

In [ ]:
person.name, person.age

### Metaclasses

A **metaclass** in Python is a class of a class. It defines how a class behaves, similar to how a class defines how an instance behaves. In simple terms, a metaclass controls the creation and behavior of classes.

In [ ]:
class A:
    pass

In [ ]:
for t in int, str, list, set, tuple, A, ValueError, type:
    print(t.__class__)

Classes are instances of `type` by default, but they all are subclasses of `object`.

![Type and object relationships](resources/type-object-relationships.png)

### `__new__` method

Metaclasses customize a class through `__new__` method. The `__new__` method in a metaclass is responsible for creating the class before it gets initialized. It allows customization of class creation, modifying attributes, methods, or even rejecting class definitions.

The` __new__` method in a metaclass receives:

- `meta_cls`: The metaclass itself.
- `cls_name`: The name of the class being created.
- `bases`: A tuple of base classes.
- `cls_dict`: A dictionary containing the class attributes and methods.

It must return a new class object using `super().__new__()`.

In [ ]:
class A:
    pass

# is equivalent to

class A(metaclass=type):
    pass

#### Basic example

In [ ]:
class MyMeta(type):
    def __new__(cls, name, bases, class_dict, **kwargs):
        print(f"Creating class: {name=}")
        print(f"{cls=}")
        print(f"{bases=}")
        print(f"{class_dict=}")
        print(f"{kwargs=}")
        return super().__new__(cls, name, bases, class_dict)

In [ ]:
class MyClass(metaclass=MyMeta, class_param_1=42, class_param_2=True):
    pass

#### Adding an attribute 

In [ ]:
class AutoAttrMeta(type):
    def __new__(cls, name, bases, class_dict):
        class_dict["auto_attr"] = "I was added by metaclass!"

        return super().__new__(cls, name, bases, class_dict)

In [ ]:
class MyClass(metaclass=AutoAttrMeta):
    pass

In [ ]:
MyClass.auto_attr

#### Class name validation

In [ ]:
class NoUnderscoreNamesMeta(type):
    def __new__(cls, name, bases, class_dict):
        if "_" in name:
            raise TypeError("Class names cannot contain underscores!")
        return super().__new__(cls, name, bases, class_dict)

In [ ]:
class ValidClass(metaclass=NoUnderscoreNamesMeta):
    pass

In [ ]:
class Invalid_Class(metaclass=NoUnderscoreNamesMeta):
    pass

### `Checked` with a metaclass

If we want to add `__slots__` to our `Checked` classes (maybe to reduce memory usage and gain performance), we can use a metaclass.

In [ ]:
from collections.abc import Callable
from typing import Any, NoReturn, get_type_hints

In [ ]:
class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        # Each descriptor instance is a class attribute, and now we need separate per-instance storage attributes.
        self.storage_name = '_' + name
        self.constructor = constructor

    def __get__(self, instance, owner=None):
        # We return either the descriptor instance or the value stored in the attribute named storage_name
        if instance is None:
            return self
        return getattr(instance, self.storage_name)

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        setattr(instance, self.storage_name, value) 

In [ ]:
class CheckedMeta(type):
    def __new__(meta_cls, cls_name, bases, cls_dict):
        if '__slots__' not in cls_dict:
            slots = []
            # class does not exist yet, that's why we cannot use typing.get_type_hints
            type_hints = cls_dict.get('__annotations__', {})
            for name, constructor in type_hints.items():
                field = Field(name, constructor)
                cls_dict[name] = field
                slots.append(field.storage_name)

            cls_dict['__slots__'] = slots

        return super().__new__(meta_cls, cls_name, bases, cls_dict) 

In [ ]:
class Checked(metaclass=CheckedMeta):
    __slots__ = ()  # skip CheckedMeta.__new__ processing

    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            self.__flag_unknown_attrs(*kwargs)

    def __flag_unknown_attrs(self, *names: str) -> NoReturn:
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

In the `Checked` class:

1. Added an empty `__slots__` to signal to `CheckedMeta.__new__` that this class doesn’t require special processing.
2. Removed `__init_subclass__`. Its job is now done by `CheckedMeta.__new__`.
3. Removed `__setattr__`. It became redundant because adding `__slots__` to the user-defined class prevents setting undeclared attributes.

In [ ]:
class Person(Checked):    
    name: str
    age: int
    salary: float

In [ ]:
person = Person(name="John Doe", age=42, salary=199_999.99)
person

In [ ]:
person.name

In [ ]:
person.name = "Jane Dane"

In [ ]:
person.year = "text"

In [ ]:
person.__dict__

In [ ]:
Person.__dict__

### `__prepare` method

The __prepare__ method is a special class method that is used in the metaclass mechanism. The purpose of `__prepare__` is to return the initial namespace (a dictionary-like object) in which the class body will be executed. The `__prepare__` method should be implemented as a `classmethod`. The namespace returned by `__prepare__` is passed in to `__new__`, but when the final class object is created the namespace is copied into a new `dict`.

In [ ]:
class Meta(type):
    @classmethod
    def __prepare__(meta_cls, cls_name, bases, **kwargs):
        print(f"Preparing namespace for {cls_name}")
        print(f"{meta_cls=}")
        print(f"{bases=}")
        print(f"{meta_cls=}")
        return {}  # You can return any mapping type

    def __new__(meta_cls, cls_name, bases, cls_dict, **kwargs):
        print(f"Creating class {cls_name}")
        return super().__new__(meta_cls, cls_name, bases, cls_dict)  

In [ ]:
class MyClass(metaclass=Meta):
    attr1 = 10
    attr2 = 20

MyClass.attr1

#### Example with `OrderedDict`

In [ ]:
from collections import OrderedDict

class Meta(type):
    @classmethod
    def __prepare__(meta_cls, cls_name, bases):
        return OrderedDict()  # Ensures attributes maintain order

    def __new__(meta_cls, cls_name, bases, cls_dict):
        print("Attributes in order:", list(cls_dict.keys()))
        return super().__new__(meta_cls, cls_name, bases, dict(cls_dict))

class MyClass(metaclass=Meta):
    a = 1
    b = 2
    c = 3

### Metaclasses in the standard library

- `abc.ABCMeta`
- `enum.EnumMeta`
- `collections.NamedTupleMeta`

We usually don't need to use these metaclasses in our code. In general, it is better to make metaclasses as implementation details, so the users of your code can easily make sense of the program. Furthermore, it is good practice to consider if a metaclass is needed at all in your use cases.

In [ ]:
from abc import ABCMeta

from collections.abc import Iterable

In [ ]:
ABCMeta.__class__

In [ ]:
Iterable.__class__

In [ ]:
issubclass(ABCMeta, type)

In [ ]:
issubclass(Iterable, type)

Every class is an instance of `type`, directly or indirectly, but metaclasses are also subclasses of `type`.

## Memory management

Python manages its own segment of memory called the **private heap**. All Python objects and data structures are stored in this private heap. The operating system manages the underlying virtual memory, but Python itself takes large chunks of memory from the OS and then manages the allocation of smaller pieces within those chunks for Python objects. Crucially, the Python interpreter itself manages this heap, meaning the memory isn't usually returned directly to the operating system when an object is deallocated. Instead, it's often kept available within the Python process for future object allocations.

**Memory management** is the process by which applications read and write data. A memory manager determines where to put an application’s data. Since there’s a finite chunk of memory, the manager has to find some free space and provide it to the application. This process of providing memory is generally called *memory allocation*.

Somewhere in the computer, there’s a physical device storing data when you’re running your Python programs. There are many layers of abstraction that the Python code goes through before the objects actually get to the hardware though. One of the main layers above the hardware (such as RAM or a hard drive) is the operating system (OS). It carries out (or denies) requests to read and write memory.

Above the OS, there are applications, one of which is the default Python implementation. **Memory management for your Python code is handled by the Python application**. The default Python implementation, **CPython**, is actually written in the C programming language. It converts Python code into instructions that it then runs on a virtual machine.

Python code actually gets compiled down to more computer-readable instructions called *bytecode*. These instructions get interpreted by a *virtual machine* when you run your code.

Okay, so **CPython is written in C, and it interprets Python bytecode**. What does this have to do with memory management? Well, the memory management algorithms and structures exist in the CPython code, in C. To understand the memory management of Python, you have to get a basic understanding of CPython itself.

CPython is written in C, which does not natively support object-oriented programming. Because of that, there are quite a bit of interesting designs in the CPython code.

#### [PyObject](https://docs.python.org/3/c-api/structures.html#c.PyObject)

<div style="border: 2px solid black; width: 200px; text-align: center; margin: 0 auto;">
    <div style="background-color: #add8e6; color: black; padding: 8px; font-weight: bold; ">PyObject</div>
    <div style="background-color: #ffffe0; border-bottom: 1px solid black; padding: 10px;">ob_refcnt</div>
    <div style="background-color: #ffffe0; padding: 10px;">ob_type</div>
</div>

<br>

You may have heard that everything in Python is an *object*, even types such as `int` and `str`. Well, it’s true on an implementation level in CPython. <mark>There is a `struct` called a **PyObject**, which every other object in CPython uses.</mark>
> A struct, or structure, in C is a custom data type that groups together different data types. To compare to object-oriented languages, it’s like a class with attributes and no methods.


The PyObject contains only two things:
- `ob_refcnt`: reference count
- `ob_type`: pointer to another type

The reference count is used for garbage collection. Then you have a pointer to the actual object type. That object type is just another struct that describes a Python object (such as a `dict` or `int`).

**Each object has its own object-specific memory allocator that knows how to get the memory to store that object. Each object also has an object-specific memory deallocator that “frees” the memory once it’s no longer needed.**

### Names in Python

**Python does not have variables. It has names.**

Let
```python
x = 2337
```

The above code is broken down into several distinct steps during execution:
1. Create a `PyObject`
2. Set the typecode to integer for the `PyObject`
3. Set the value to `2337` for the `PyObject`
4. Create a name called `x`
5. Point `x` to the new `PyObject`
6. Increase the refcount of the `PyObject` by `1`


> When you create an integer object, like `x = 2337`, the value of the integer is stored within a specialized structure that extends the basic `PyObject` structure. For integers, this structure is typically [`PyLongObject` in CPython](https://docs.python.org/3/c-api/long.html#integer-objects), the reference implementation of Python. The `PyLongObject` structure contains the basic `PyObject` fields and *an additional field to store the integer value*.

In memory, it might looks something like this:
<div style="text-align: center;">
    <img src="https://files.realpython.com/media/py_memory1.2b6e5f8e5bc9.png" alt="Index" width="500" height="500"/>
</div>

If you were to try to assign a new value to `x`, you could try the following:
```python
x = 2338
```
This code:
1. Creates a new `PyObject`
2. Sets the typecode to integer for the `PyObject`
3. Sets the value to `2338` for the `PyObject`
4. Points `x` to the new `PyObject`
5. Increases the refcount of the new `PyObject` by `1`
6. Decreases the refcount of the old `PyObject` by `1`

<div style="text-align: center;">
    <img src="https://files.realpython.com/media/py_memory2.99bb432c3432.png" alt="Index" width="500" height="500"/>
</div>

This diagram helps illustrate that `x` points to a reference to an object and doesn't own the memory space as before. It also shows that the `x = 2338` command is not an assignment, but rather binding the name `x` to a reference.

```python
y = x
```
<div style="text-align: center;">
    <img src="https://files.realpython.com/media/py_memory3_1.ea43471d3bf6.png" alt="Index" width="500" height="500"/>
</div>

Now you can see that a new Python object has not been created, just a new name that points to the same object. Also, the object’s `refcount` has increased by one.

```python
y += 1
```
<div style="text-align: center;">
    <img src="https://files.realpython.com/media/py_memory4.0a15e8415a15.png" alt="Index" width="600" height="600"/>
</div>

After the addition call, you are returned with a new Python object. A new object has been created, and `y` now points to the new object.

Python pre-creates a certain subset of objects in memory and keeps them in the global namespace for everyday use.

Which objects depend on the implementation of Python. CPython 3.7 interns the following:

1. Integer numbers between -5 and 256
2. Strings that are less than 20 characters and contain ASCII letters (A to Z and a to z), digits, and underscores only

The reasoning behind this is that these objects are likely to be used in many programs. For example, most variable names in Python are covered by the second bullet point. By interning these objects, Python prevents memory allocation calls for consistently used objects.

In [ ]:
x = 42
y = 42

x is y

In [ ]:
x = "hello"
y = "hello"

x is y

In [ ]:
x = 4224
y = 4224

x is y

### CPython’s Memory Management: Allocating memory for objects

As we know the operating system (OS) abstracts the physical memory and creates a virtual memory layer that applications (including Python) can access.
<div style="text-align: center;">
    <img src="https://www.cs.uic.edu/~jbell/CourseNotes/OperatingSystems/images/Chapter9/9_01_VirtualMemoryLarger.jpg" alt="Index" width="600" height="600"/>
</div>

An OS-specific virtual memory manager carves out a chunk of memory for the Python process. The darker gray boxes in the image below are now owned by the Python process.
<div style="text-align: center;">
    <img src="https://files.realpython.com/media/memory_management.92ad564ec680.png" alt="Index" width="700" height="700"/>
</div>

Python uses a portion of the memory for internal use and non-object memory. The other portion is dedicated to object storage (your `int`, `dict`, and the like). 

**CPython has an object allocator that is responsible for allocating memory within the object memory area**. This object allocator is where most of the magic happens. It gets called every time a new object needs space allocated or deleted. Typically, the adding and removing of data for Python objects like list and int doesn’t involve too much data at a time. So the design of the allocator is tuned to work well with small amounts of data at a time. It also tries not to allocate memory until it’s absolutely required.

> The comments in the [source code](https://github.com/python/cpython/blob/7d6ddb96b34b94c1cbdf95baa94492c48426404e/Objects/obmalloc.c) describe the allocator as “a fast, special-purpose memory allocator for small blocks, to be used on top of a general-purpose malloc.” In this case, malloc is C’s library function for memory allocation.

Now we’ll look at CPython’s memory allocation strategy. First, we’ll talk about the 3 main pieces and how they relate to each other.

**Arenas** are the largest chunks of memory and are aligned on a page boundary in memory. A page boundary is the edge of a fixed-length contiguous chunk of memory that the OS uses. Python assumes the system’s page size is 256 kilobytes.
<div style="text-align: center;">
    <img src="https://files.realpython.com/media/memory_management_5.394b85976f34.png" alt="Index" width="800" height="800"/>
</div>

Within the arenas are **pools**, which are one virtual memory page (4 kilobytes). These pools are fragmented into smaller **blocks** of memory.

**All the blocks in a given pool are of the same “size class”**. A size class defines a specific block size, given some amount of requested data. The chart below is taken directly from the source code comments:
| Request in bytes | Size of allocated block | Size class idx |
| :--------------: | :---------------------: | :------------: |
|      1-8         |            8            |       0        |
|      9-16        |           16            |       1        |
|     17-24        |           24            |       2        |
|     25-32        |           32            |       3        |
|     33-40        |           40            |       4        |
|     41-48        |           48            |       5        |
|     49-56        |           56            |       6        |
|     57-64        |           64            |       7        |
|     65-72        |           72            |       8        |
|       ...        |           ...           |      ...       |
|    497-504       |          504            |      62        |
|    505-512       |          512            |      63        |

For example, if 42 bytes are requested, the data would be placed into a size 48-byte block.

##### **Pools**
Pools are composed of blocks from a single size class. Each pool maintains a double-linked list to other pools of the same size class. In that way, the algorithm can easily find available space for a given block size, even across different pools.

A `usedpools` list tracks all the pools that have some space available for data for each size class. When a given block size is requested, the algorithm checks this `usedpools` list for the list of pools for that block size.

Pools themselves must be in one of 3 states: **used**, **full**, or **empty**. A used pool has available blocks for data to be stored. A full pool’s blocks are all allocated and contain data. An empty pool has no data stored and can be assigned any size class for blocks when needed.

A `freepools` list keeps track of all the pools in the empty state. But when do empty pools get used?

**Assume your code needs an 8-byte chunk of memory. If there are no pools in `usedpools` of the 8-byte size class, a fresh empty pool is initialized to store 8-byte blocks. This new pool then gets added to the `usedpools` list so it can be used for future requests**.

Say a full pool frees some of its blocks because the memory is no longer needed. That pool would get added back to the `usedpools` list for its size class.

You can see now how pools can move between these states (and even memory size classes) freely with this algorithm.

##### **Blocks**
<div style="text-align: center;">
    <img src="https://files.realpython.com/media/memory_management_3.52bffbf302d3.png" alt="Index" width="800" height="800"/>
</div>

As seen in the diagram above, pools contain a pointer to their “free” blocks of memory. There’s a slight nuance to the way this works. This allocator “strives at all levels (arena, pool, and block) never to touch a piece of memory until it’s actually needed,” according to the comments in the source code.

That means that a pool can have blocks in 3 states. These states can be defined as follows:
- **untouched**: a portion of memory that has not been allocated
- **free**: a portion of memory that was allocated but later made “free” by CPython and that no longer contains relevant data
- **allocated**: a portion of memory that actually contains relevant data

**The `freeblock` pointer points to a singly linked list of free blocks of memory. In other words, a list of available places to put data. If more than the available free blocks are needed, the allocator will get some untouched blocks in the pool**.

As the memory manager makes blocks “free,” those now free blocks get added to the front of the freeblock list. The actual list may not be contiguous blocks of memory, like the first nice diagram. It may look something like the diagram below:
<div style="text-align: center;">
    <img src="https://files.realpython.com/media/memory_management_4.4a30dfa2d111.png" alt="Index" width="800" height="800"/>
</div>

##### **Arenas**
Arenas contain pools. Those pools can be used, full, or empty. Arenas themselves don’t have as explicit states as pools do though.

**Arenas are instead organized into a doubly linked list called `usable_arenas`. The list is sorted by the number of free pools available. The fewer free pools, the closer the arena is to the front of the list**.
<div style="text-align: center;">
    <img src="https://files.realpython.com/media/memory_management_6.60e9761bc158.png" alt="Index" width="800" height="800"/>
</div>

This means that the arena that is the most full of data will be selected to place new data into. But why not the opposite? Why not place data where there’s the most available space?

This brings us to the idea of truly freeing memory. The reason is that when a block is deemed “free”, that memory is not actually freed back to the operating system. The Python process keeps it allocated and will use it later for new data. Truly freeing memory returns it to the operating system to use.

Arenas are the only things that can truly be freed. So, it stands to reason that those arenas that are closer to being empty should be allowed to become empty. That way, that chunk of memory can be truly freed, reducing the overall memory footprint of your Python program.


**How They Work Together**

**Allocation**: When a new object needs to be created, CPython first determines the size class of the object. It then looks for a pool that handles objects of that size. If there is an available block within the pool, it is allocated to the object. If not, a new pool may be created within an arena, or a new arena may be allocated if necessary.

**Deallocation**: When an object is no longer needed and is to be destroyed, its block is deallocated and returned to the pool for reuse. This process is managed by the Python garbage collector, which also handles detecting and collecting cyclic references that cannot be freed by reference counting alone.

<mark>The use of arenas reduces the number of system calls for memory allocation, as a single large block is requested from the operating system less frequently than many small blocks would be. This bulk allocation strategy reduces overhead and fragmentation at the OS level.</mark>

<mark>By segregating objects into size classes, pools help to minimize internal fragmentation (unused space within allocated memory areas). This also allows for rapid allocation and deallocation of objects, as a pool can quickly find a free block without having to search or fit different-sized objects.</mark>


***

### Garbage collection

**Garbage collection (GC)** is nothing but collecting or gaining memory back which has been allocated to objects but which is not currently in use in any part of our program. It is the process in which programs try to free up memory space that is no longer used by objects.

Garbage collection is implemented differently for every language. Most high-level programming languages have some sort of garbage collection built in. Low-level programming languages may add garbage collection through libraries.

In C programming, developers need to take care of memory allocation and deallocation using `malloc()` and `free()` functions. But, in the case of C# developers don't need to take care of GC and it’s not recommended either. In C/C++ the developer must manually delete objects and free up memory. Relying on manual processes made it easy to introduce bugs into the code, some of which can have serious consequences. For example, a developer might forget to free up memory after the program no longer needs it, leading to a **memory leak** that quickly consumes all the available RAM. Or the developer might free up an object's memory space without modifying a corresponding pointer, resulting in a **dangling pointer** that causes the application to be buggy or even to crash.

Programming languages that include garbage collection try to eliminate these types of bugs by using carefully designed GC algorithms to control memory deallocation. The garbage collector automatically detects when an object is no longer needed and removes it, freeing up the memory space allocated to that object without affecting objects that are still being used.

### Strategies

#### Tracing
Tracing garbage collection is the predominant form of garbage collection, to the extent that the term "garbage collection" is frequently synonymous with this method rather than alternatives like reference counting. The fundamental approach involves identifying objects for garbage collection by tracing a path of references from specific root objects to determine which ones are accessible. Objects not reachable in this way are deemed garbage and are subsequently collected. Nonetheless, the actual implementation of this strategy varies significantly, employing numerous algorithms that differ greatly in their complexity and performance traits.
<div style="text-align: center;">
    <img src="https://miro.medium.com/v2/resize:fit:840/1*GN770FzmaWwTVdx94GneHQ.gif" alt="Index" width="400" height="400"/>
</div>

#### Reference counting
Reference counting garbage collection is where each object has a count of the number of references to it. Garbage is identified by having a reference count of zero. An object's reference count is incremented when a reference to it is created, and decremented when a reference is destroyed. When the count reaches zero, the object's memory is reclaimed.
<div style="text-align: center;">
    <img src="https://book.huihoo.com/data-structures-and-algorithms-with-object-oriented-design-patterns-in-java/html/img1704.gif" alt="Index" width="700" height="700"/>
</div>

However, with there are a few problems that need to be addressed:
- **Cycles:** If two or more objects refer to each other, they can create a cycle whereby neither will be collected as their mutual references never let their reference counts become zero. Some garbage collection systems using reference counting (like the one in CPython) use specific cycle-detecting algorithms to deal with this issue.
- **Space overhead (reference count):** Reference counting requires space to be allocated for each object to store its reference count. The count may be stored adjacent to the object's memory or in a side table somewhere else, but in either case, every single reference-counted object requires additional storage for its reference count. Memory space with the size of an unsigned pointer is commonly used for this task, meaning that **32 or 64 bits of reference count storage must be allocated for each object**. On some systems, it may be possible to mitigate this overhead by using a tagged pointer to store the reference count in unused areas of the object's memory. Often, an architecture does not actually allow programs to access the full range of memory addresses that could be stored in its native pointer size; certain number of high bits in the address is either ignored or required to be zero.
- **Space overhead (increment/decrement):** In naive implementations, each assignment of a reference and each reference falling out of scope often require modifications of one or more reference counters. However, in a common case when a reference is copied from an outer scope variable into an inner scope variable, such that the lifetime of the inner variable is bounded by the lifetime of the outer one, the reference incrementing can be eliminated. The outer variable "owns" the reference.
- **Atomicity:** When used in a multithreaded environment, these modifications (increment and decrement) may need to be atomic operations such as compare-and-swap, at least for any objects which are shared, or potentially shared among multiple threads. Atomic operations are expensive on a multiprocessor, and even more expensive if they have to be emulated with software algorithms. It is possible to avoid this issue by adding per-thread or per-CPU reference counts and only accessing the global reference count when the local reference counts become or are no longer zero (or, alternatively, using a binary tree of reference counts, or even giving up deterministic destruction in exchange for not having a global reference count at all), but this adds significant memory overhead and thus tends to be only useful in special cases (it is used, for example, in the reference counting of Linux kernel modules).
- **Real-time:** Naive implementations of reference counting do not generally provide real-time behavior, because any pointer assignment can potentially cause a number of objects bounded only by total allocated memory size to be recursively freed while the thread is unable to perform other work. It is possible to avoid this issue by delegating the freeing of unreferenced objects to other threads, at the cost of extra overhead.

In [ ]:
import sys

def get_num_refs(obj):
    """
    Returns the approximate number of references to the given object `obj`.

    Note:
        `sys.getrefcount(obj)` includes:
        - The temporary reference passed to getrefcount
        - The reference created when passing obj to get_num_refs
        Hence, we subtract 2 to get a more accurate count.

    Args:
        obj (Any): The object to check reference count for.

    Returns:
        int: Approximate number of references to the object.
    """
    return sys.getrefcount(obj) - 2

The reference count gets increased for a few different reasons. For example, the reference count will increase if you assign it to another variable:

In [ ]:
numbers = [1, 2, 3]
get_num_refs(numbers)

In [ ]:
more_numbers = numbers
get_num_refs(numbers)

It will also increase if you pass the object as an argument.

In [ ]:
total = sum(numbers)
get_num_refs(numbers)

As a final example, the reference count will increase if you include the object in a list:

In [ ]:
matrix = [numbers, numbers, numbers]
get_num_refs(numbers)

Python allows you to inspect the current reference count of an object with the `sys` module. You can use `sys.getrefcount(numbers)`, but keep in mind that passing in the object to `getrefcount()` increases the reference count by 1. See the implementation of `get_num_refs` function above.
> Initially, you have your reference to `numbers`. When you pass numbers to `sys.getrefcount(numbers)`, `sys.getrefcount()` creates a temporary reference to `numbers` for the duration of the function call. This is why the reference count appears to increase by 1.
After the `sys.getrefcount()` call, the temporary reference created by the function is removed.
So, when you subtract 1 from the result of `sys.getrefcount(numbers)`, you are essentially accounting for the temporary reference created by the function, giving you the actual reference count of `numbers`.
> 
> The reason this doesn't happen with the `sum()` function is because `sum()` does not create any additional references to the objects it operates on, including the numbers list. It simply iterates over the elements of the iterable (in this case, the `list`), performs the addition, and returns the result.

<mark>In any case, if the object is still required to hang around in your code, its reference count is greater than 0. Once it drops to 0, the object has a specific deallocation function that is called which “frees” the memory so that other objects can use it.</mark>

### The `gc` module
The `gc` (garbage collector) package in Python is a module that provides an interface to the garbage collection facility in the Python interpreter. Its primary role is to manage the automatic memory management of Python objects, particularly focusing on the detection and collection of cyclic garbage.

**In general, you do not need to manually interact with the garbage collector**. Python's automatic garbage collection is sufficient for most applications. However, in memory-critical applications or when dealing with large data sets, manual control can be beneficial to optimize performance or to debug.

In some scenarios, especially in a multithreaded environment where the garbage collection might introduce unwanted pauses, developers might choose to disable the automatic garbage collection and manually trigger it at more convenient times.

An example code demonstrating the usage of the module:

In [ ]:
import gc

In [ ]:
class MyClass:
    def __init__(self):
        self._my_ref = None

    def set_ref(self, obj):
        self._my_ref = obj

    def __del__(self):
        print(f"MyClass instance {id(self)} has been destroyed")


# Disable automatic garbage collection to manually control it.
gc.disable()

# Create a reference cycle.
obj1 = MyClass()
obj2 = MyClass()

obj1.set_ref(obj2)
obj2.set_ref(obj1)

In [ ]:
get_num_refs(obj1), get_num_refs(obj2)

This cycle prevents Python's reference counting system from deallocating them when their external references are deleted (with `del` statements).

`del` statement doesn’t delete objects, it removes the name (and reference) to the object. When the reference count is zero, the object is deleted from the system by the garbage collection.

In [ ]:
# Delete references.
del obj1
del obj2

In [ ]:
# Explicitly run garbage collection.
collected = gc.collect()

# This function returns the number of objects it successfully collected and deallocated.
collected

In [ ]:
# Enable automatic garbage collection.
gc.enable()

For some applications, especially those that create and destroy many objects quickly, disabling garbage collection temporarily can improve performance. This is because the overhead of the garbage collector running can be more significant than the benefit of the memory it recovers during intensive operations. However, this strategy should be used cautiously, as it can lead to increased memory usage if garbage collection is not re-enabled in a timely manner.

Here's an example to illustrate the performance difference when disabling garbage collection. We'll use a simple case where many objects are created, and compare the execution time with and without garbage collection:

In [ ]:
import time


N = 1_000_000


def create_objects(count):
    for _ in range(count):
        a = [0] * 10000  # Creating a large object.


def with_gc():
    start_time = time.perf_counter()
    create_objects(N)  # Create N objects.
    end_time = time.perf_counter()
    
    print(f"Time with GC enabled: {end_time - start_time:.3f} seconds")


def without_gc():
    gc.disable()  # Disable garbage collection.
    
    start_time = time.perf_counter()
    create_objects(N)  # Create N objects.
    end_time = time.perf_counter()
    
    gc.enable()  # Re-enable garbage collection
    print(f"Time with GC disabled: {end_time - start_time:.3f} seconds")


with_gc()
without_gc()

### References
- [Garbage collection](https://en.wikipedia.org/wiki/Garbage_collection_(computer_science))
- [Memory Management](https://docs.python.org/3/c-api/memory.html)
- [Memory Management in Python](https://realpython.com/python-memory-management/)
- [Pointers in Python: What's the Point?](https://realpython.com/pointers-in-python/)
- [`gc` module](https://docs.python.org/3/library/gc.html#module-gc)